In [46]:
!pip install  mediapipe
import mediapipe as mp

In [68]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1)
mp_draw = mp.solutions.drawing_utils

In [69]:
import cv2
import mediapipe as mp
import numpy as np
#pip install pygame
import pygame
import math

In [70]:
pygame.mixer.init()

In [74]:
circle_sound = pygame.mixer.Sound("circle_sound.ogg")  # Replace with your sound file
rectangle_sound = pygame.mixer.Sound("rectangle_sound.ogg") #  # Replace with your sound file
eraser_sound = pygame.mixer.Sound("eraser_sound.wav")

In [75]:
cap = cv2.VideoCapture(0)
w, h = 1280, 720
cap.set(3, w)
cap.set(4, h)

True

In [76]:
canvas = np.zeros((h, w, 3), np.uint8)

In [77]:
def distance(p1, p2):
    return math.hypot(p2[0] - p1[0], p2[1] - p1[1])

In [78]:
tools = ['Circle', 'Rectangle', 'Eraser']
selected_tool = None
tool_colors = [(0, 255, 0), (255, 0, 0), (100, 100, 100)]
tool_height = 100


In [79]:

while True:
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)

    # Draw tool selection bar
    for i, tool in enumerate(tools):
        x1 = i * (w // 3)
        x2 = (i + 1) * (w // 3)
        y1 = 0
        y2 = tool_height
        color = (0, 255, 255) if selected_tool == tool else tool_colors[i]
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, -1)
        cv2.putText(frame, tool, (x1 + 40, 70), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 0), 3)

    if results.multi_hand_landmarks:
        for handLms in results.multi_hand_landmarks:
            lm_list = []
            for lm in handLms.landmark:
                cx, cy = int(lm.x * w), int(lm.y * h)
                lm_list.append((cx, cy))

            mp_draw.draw_landmarks(frame, handLms, mp_hands.HAND_CONNECTIONS)

            index_tip = lm_list[8]
            middle_tip = lm_list[12]
            thumb_tip = lm_list[4]

            # Tool selection
            if index_tip[1] < tool_height:
                tool_index = index_tip[0] // (w // 3)
                if 0 <= tool_index < len(tools):
                    selected_tool = tools[tool_index]

            # Perform action based on selected tool
            if selected_tool == 'Circle':
                if distance(thumb_tip, index_tip) < 40:
                    cv2.circle(canvas, index_tip, 30, (0, 255, 0), 2)
                    circle_sound.play()

            elif selected_tool == 'Rectangle':
                if index_tip[1] < lm_list[6][1] and middle_tip[1] < lm_list[10][1]:
                    pt1 = (index_tip[0] - 30, index_tip[1] - 30)
                    pt2 = (middle_tip[0] + 30, middle_tip[1] + 30)
                    cv2.rectangle(canvas, pt1, pt2, (255, 0, 0), 2)
                    rectangle_sound.play()

            elif selected_tool == 'Eraser':
                if index_tip[1] < lm_list[6][1] and middle_tip[1] > lm_list[10][1]:
                    cv2.circle(canvas, index_tip, 40, (0, 0, 0), -1)
                    eraser_sound.play()

    # Merge canvas and frame
    combined = cv2.addWeighted(frame, 0.7, canvas, 0.3, 0)
    cv2.imshow("AI Virtual Drawing Plane", combined)
    cv2.putText(frame, "AI Virtual Drawing Board", (w//2 - 300, tool_height + 50),
    cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255), 3)

    key = cv2.waitKey(1)
    if key == ord('q'):
        break
    elif key == ord('c'):
        canvas = np.zeros((h, w, 3), np.uint8)

cap.release()
cv2.destroyAllWindows()